In [ ]:
import numpy as np
import math
import pickle
import itertools
import time
import datetime
import cvxpy as cvx
import mosek
import copy

import mkl
import pickle
import os
import ray
import warnings
import psutil
warnings.filterwarnings("ignore")

from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
from numpy import transpose as trans
from collections import OrderedDict

import subprocess
subprocess.call("bash convert_files.sh", shell=True)
from auxiliary import is_pos_def, cond, rotate_matrix,  gen_train_data, gen_test_data
from datasets import load_parkinson, load_triazines,  load_wine, load_fertility, load_forest_fires

In [ ]:
ray.init(object_store_memory=int(5e10), num_cpus=48,  redis_password="password54322423")

In [ ]:
def fit_linear(train_data, fit_intercept):
    
    """ LR fitter. fit_intercept determines whether or not to fit the y-intercept in the regression. This set to false by default. """

    X_train, y_train = train_data
    n, p = X_train.shape

    from sklearn.linear_model import LinearRegression
        
    warnings.filterwarnings("ignore")
    lr = LinearRegression(fit_intercept=fit_intercept)
    lr.fit(X_train, y_train)
        
    return lr

In [ ]:
def fit_lasso(train_data, sigma, cv, fit_intercept, alpha_scaling, n_folds=5):
    
    """ Lasso fitter. If cv True uses CV to fit; if false will use alpha_scaling * \sqrt{2 log p/n} as a regularizer
    fit_intercept determines whether or not to fit the y-intercept in the regression. This set to false by default. """

    X_train, y_train = train_data
    n, p = X_train.shape

    from sklearn.linear_model import Lasso, LassoCV
        
    warnings.filterwarnings("ignore")

    # Theoretically Optimal regularization and CV regularizers

    if not cv:
        alpha = alpha_scaling*sigma*math.sqrt(2* (math.log(p)/n))
        lasso = Lasso(alpha=alpha, max_iter=5000, fit_intercept=fit_intercept)
        lasso.fit(X_train, y_train)
    else:
        alphas = np.logspace(-6, 1, num=100)
        lasso=LassoCV(max_iter=5000, cv=n_folds, alphas=alphas, fit_intercept=fit_intercept)
        
        # Run LassoCV with the metric for CV as MSE
        lasso.fit(X_train, y_train)
        
    return lasso

In [ ]:
def fit_ridge(train_data, sigma, cv, fit_intercept, alpha_scaling=1.0):
    
    """ ridge regression fitter. If cv True uses CV to fit; if false will use alpha_scaling as a regularizer
    fit_intercept determines whether or not to fit the y-intercept in the regression. This set to false by default. """

    X_train, y_train = train_data
    n, p = X_train.shape

    from sklearn.linear_model import Ridge, RidgeCV
        
    warnings.filterwarnings("ignore")

    # Theoretically Optimal regularization and CV regularizers

    if not cv:
        alpha = alpha_scaling
        ridge = Ridge(alpha=alpha, fit_intercept=fit_intercept)
        ridge.fit(X_train, y_train)
    else:
        alphas = np.logspace(-6, 1, num=100)
        ridge=RidgeCV(alphas=alphas, fit_intercept=fit_intercept)
        
        # Run LassoCV with the metric for CV as MSE
        ridge.fit(X_train, y_train)

    return ridge

In [ ]:
def fit_elastic(train_data, sigma, cv, fit_intercept, alpha_scaling=1.0, n_folds=5):
    
    """ elastic net fitter. If cv True uses CV to fit; if false will use alpha_scaling as a regularizer
    fit_intercept determines whether or not to fit the y-intercept in the regression. This set to false by default. """
    
    X_train, y_train = train_data
    n, p = X_train.shape

    from sklearn.linear_model import ElasticNet
    warnings.filterwarnings("ignore")

    # Theoretically Optimal regularization and CV regularizers
    
    if not cv:
        alpha = alpha_scaling*sigma*math.sqrt(2*(math.log(p)/n))
        elastic = ElasticNet(alpha=alpha, max_iter=5000, fit_intercept=fit_intercept)
        elastic.fit(X_train, y_train)
    else:
        alphas = np.logspace(-6, 1, num=100)
        ratio = [.1, .5, .7, .9, .95, .99, 1]
        elastic=ElasticNetCV(max_iter=5000, cv=n_folds, l1_ratio = ratio, alphas=alphas, fit_intercept=fit_intercept)
        
        # Run LassoCV with the metric for CV as MSE
        elastic.fit(X_train, y_train)

    return elastic

In [ ]:
@ray.remote
def invert(X, x_star, lam, threads=1):

    """Parallel Function to solve JM program"""
    
    warnings.filterwarnings("ignore")
    mkl.set_num_threads(threads)
    try:
        n, p = X.shape
        eps = 1e-12
        Sigma_n = 1/float(n) * np.transpose(X) @ X  + eps*np.eye(p)
        #adding epsilon to make matrix strictly p.s.d. or else cvxpy throws not DCP error (cannot recognize convexity of objective)

        w = cvx.Variable(p)
        obj = cvx.Minimize(cvx.quad_form(w, Sigma_n))
        const = [cvx.norm(Sigma_n * w  - x_star, "inf") <= lam]

        prob = cvx.Problem(obj, const)
        sol = prob.solve(solver=cvx.MOSEK, mosek_params={mosek.iparam.num_threads: threads})
        
        return sol, w.value
    except:
        # If solver fails simply return None value for w
        return math.inf, None

In [ ]:
def compute_ws(X_train, X_test, lams, threads):
    
    """ Computes the entire set of w's needed for X_test using X_train """
    
    warnings.filterwarnings("ignore")
    train_n, p = X_train.shape
    test_n, p = X_test.shape

    ids=[]
    lam_deb, method = lams[0], lams[1]
    for i in range(test_n):
        x_star = X_test[i, :]
        x_norm = np.linalg.norm(x_star, ord=2)
        
        # Computes Debiasing Correction using Theoretical Value of \lambda_w
        if method=="theory":
            if (train_n >= 1.5*p):
                lam_deb = .01*lam_deb
                ids.append(invert.remote(X_train, x_star, lam=lam_deb*x_norm, threads=threads))
            else:
                ids.append(invert.remote(X_train, x_star, lam=lam_deb*x_norm, threads=threads))
        # Computes Debiasing Correction for a fixed \lambda_w
        elif method=="grid":
            ids.append(invert.remote(X_train, x_star, lam=lam_deb*x_norm, threads=threads))
        else:
            raise Exception("Aux Debiasing Set Incorrectly")
    vals = ray.get(ids) # Get values from Ray
    ray.internal.free(ids) 
    
    return vals

In [ ]:
def debias_base(train_data, base_model, X_test, ws):
    
    """Computes JM Debiased Predictions with respect to the base model using base_model and learned ws"""
    
    warnings.filterwarnings("ignore")
    X_train, y_train = train_data
    train_n, p = X_train.shape
    test_n, p = X_test.shape
    
    beta = base_model.coef_
    beta_int = base_model.intercept_

    y_preds=np.zeros(test_n)
    resid = np.transpose(X_train) @ (y_train-X_train @ beta-beta_int)
    feasible=[]
    for i in range(test_n):
        x_star = X_test[i, :]
        #Lasso Prediction
        y_preds[i] = x_star.dot(beta) + beta_int
        val, w = ws[i]
        #debiasing correction
        if val==math.inf:
            feasible.append(False)
        else:
            y_preds[i] += 1/float(train_n) * np.ravel(w).dot(resid)
            feasible.append(True)
    
    return y_preds, feasible

In [ ]:
def run_JM_expt(data, main_reg, cv, fit_intercept, sigma, lams, threads):
    # run an entire experiment for a given value of p, n, s. These will not use CV 
    
    X_train, y_train, X_test, y_test = data
    
    if main_reg=="Lasso":
        f_main = fit_lasso((X_train, y_train), sigma=sigma, cv=cv, fit_intercept=fit_intercept, alpha_scaling=1.0, n_folds=5)
    elif main_reg=="Ridge":
        f_main = fit_ridge((X_train, y_train), sigma=sigma, cv=cv, fit_intercept=fit_intercept, alpha_scaling=1.0)
    elif main_reg=="Elastic":
        f_main = fit_elastic((X_train, y_train), sigma=sigma, cv=cv, fit_intercept=fit_intercept, alpha_scaling=1.0, n_folds=5)
    else:
        raise Exception("Main Reg Set Incorrectly")    

    f_main_preds = f_main.predict(X_test)
    
    lam_vals, method = lams[0], lams[1]
    if method=="theory":
        lam_deb = 1.5*math.sqrt(math.log(p)/train_n)
        w = compute_ws(X_train, X_test, lams=(lam_deb, method), threads=threads)
        deb_pred, feasible = debias_base((X_train, y_train), f_main, X_test, ws)
        deb_preds = [deb_pred]
        feasibles = [feasible]
        ws = [w]
    elif method=="grid":
        deb_preds=[]
        feasibles=[]
        ws=[]
        for lam_deb in lam_vals:
            w = compute_ws(X_train, X_test, lams=(lam_deb, method), threads=threads)
            deb_pred, feasible = debias_base((X_train, y_train), f_main, X_test, w)
            deb_preds.append(deb_pred)
            feasibles.append(feasible)
            
    kappa=cond(X_train)
    
    return deb_preds, f_main_preds, ws, y_test, kappa, feasibles

In [ ]:
def run_LinReg_expt(data, fit_intercept):
    # run an entire experiment for a given value of p, n, s. These will not use CV 
    
    X_train, y_train, X_test, y_test = data
    f_main = fit_linear((X_train, y_train), fit_intercept=fit_intercept) 
    f_main_preds = f_main.predict(X_test)
     
    kappa=cond(X_train)
    
    return f_main_preds, y_test, kappa

In [ ]:
def save_JM_expts(dataset, main_reg_param, cv, fit_intercept, sigma, lams, threads, parallel_params, folder_path):

    # function to potentially parallelize experiments across various values of p, n, s and save data in pkl file
    # Runs JM estimator on Real Data
    save_data = OrderedDict()
    
    chunk_size, threads = parallel_params
    main_reg = main_reg_param["method"]
    save_data["main_reg_params"]=main_reg_param
    save_data["dataset"]=str(dataset)
    save_data["main_reg"]="CV"+str(cv)
    save_data["fit_intercept"]=str(fit_intercept)
    save_data["output"] = "deb_preds, main_preds, ws, y_test, kappa, feasibles, mu_y"
    save_data["sigma"] = sigma
    save_data["lam"] = lams[0]
    save_data["lam_method"] = lams[1]

    if dataset=="Triazines":
        data=load_triazines(test_size=.20)
    elif dataset=="Wine":
        data=load_wine()
    elif dataset=="Parkinson":
        data=load_parkinson()
    elif dataset=="Fertility":
        data=load_fertility()
    elif dataset=="Fire":
        data=load_forest_fires()
        
    assert(fit_intercept==False, "This should be set to false for all f regression. g_lasso has been manually set to include this")
    X_train, y_train, X_test, y_test = data
    train_n, p = X_train.shape
    test_n, _ = X_test.shape
    #print(test_n)
    mu_y = np.mean(y_train)
    y_train = y_train - mu_y
    
    print("Starting")

    deb_preds=[]
    lasso_preds=[]
    y_tests=[] 
    kappas=[]
    feasibles=[]

    deb_pred, lasso_pred, w, y_test, kappa, feasible = run_JM_expt((X_train, y_train, X_test, y_test), main_reg=main_reg, cv=cv, fit_intercept=fit_intercept, sigma=sigma, lams=lams, threads=threads)
    deb_preds.append([i+mu_y for i in deb_pred])
    lasso_preds.append(lasso_pred+mu_y)
    y_tests.append(y_test)
    kappas.append(kappa)
    feasibles.append(feasible)
    
    save_data["results"] = [lams,
                deb_preds,
                lasso_preds,
                y_tests,
                kappas,
                feasibles]
        
    print("Saving Data")
    time.sleep(1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    file_name = "JM_"+str(dataset)+"_"+timestr+".pickle"
    
    file_path = os.path.join(folder_path, file_name)
    pickle.dump(save_data, open(file_path, "wb"))
    
    return save_data

In [ ]:
def save_LinReg_expts(dataset, fit_intercept, folder_path):

    # function to potentially parallelize experiments across various values of p, n, s and save data in pkl file
    # Runs Linear Regressionon on Real Datasets
    save_data = OrderedDict()
    
    save_data["main_reg_params"]="LinReg"
    save_data["dataset"]=str(dataset)
    save_data["fit_intercept"]=str(fit_intercept)
    save_data["output"] = "main_preds, y_test, kappa, mu_y"
    save_data["lam"] = lams[0]
    save_data["lam_method"] = lams[1]

    if dataset=="Triazines":
        data=load_triazines(test_size=.20)
    elif dataset=="Wine":
        data=load_wine()
    elif dataset=="Parkinson":
        data=load_parkinson()
    elif dataset=="Fertility":
        data=load_fertility()
    elif dataset=="Fire":
        data=load_forest_fires()
        
    X_train, y_train, X_test, y_test = data
    train_n, p = X_train.shape
    test_n, _ = X_test.shape
    #print(test_n)
    
    print("Starting")

    main_preds=[]
    y_tests=[] 
    kappas=[]

    main_pred, y_test, kappa = run_LinReg_expt((X_train, y_train, X_test, y_test), fit_intercept=fit_intercept)
    main_preds.append([i for i in main_pred])
    y_tests.append(y_test)
    kappas.append(kappa)
    
    save_data["results"] = [main_preds,
                y_tests,
                kappas]
        
    print("Saving Data")
    time.sleep(1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    file_name = "LinReg_"+str(dataset)+"_"+timestr+".pickle"
    
    file_path = os.path.join(folder_path, file_name)
    pickle.dump(save_data, open(file_path, "wb"))
    
    return save_data

In [ ]:
sigma=1.0
computer_cpus = psutil.cpu_count()
main_reg_params = [({"method" : "Lasso"}, 48, computer_cpus//48), ({"method" : "Ridge"}, 12, computer_cpus//12), ({"method" : "Elastic"}, 48, computer_cpus//48)]
datasets=["Fertility", "Triazines", "Fire", "Wine", "Parkinson"]
JM_aux_params=[(np.logspace(-7, 2, num=100), "grid")]
cv=True       
fit_intercept=False
path_options="JM_Real"

now = datetime.datetime.now()
folder_path=str(now.month)+"-"+str(now.day)+"-"+path_options
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
total=0
for lams in JM_aux_params:
    for dataset in datasets:
        total+=1

In [ ]:
count=0
for main_reg_param, chunk_size, threads in main_reg_params:
    for lams in JM_aux_params:
        for dataset in datasets:
            count+=1
            print("{0:.0%}".format(float(count)/total)+" Done")
            data_cv_true_JM = save_JM_expts(dataset=dataset, main_reg_param=main_reg_param, cv=cv, fit_intercept=fit_intercept, sigma=sigma, lams=lams, threads=threads, parallel_params=(chunk_size, threads), folder_path=folder_path)

In [ ]:
path_options="LinReg_Real"
now = datetime.datetime.now()
folder_path=str(now.month)+"-"+str(now.day)+"-"+path_options
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
count=0
for dataset in datasets:
    count+=1
    print("{0:.0%}".format(float(count)/total)+" Done")
    data_lin_reg = save_LinReg_expts(dataset=dataset, fit_intercept=True, folder_path=folder_path)